# Call

`Call` is a simple object for one-off calls to LLMs. It does not track history or token usage.

In [1]:
from irouter import Call
from irouter.base import nb_markdown

# To load OPENROUTER_API_KEY from .env file create a .env file at the root of the project with OPENROUTER_API_KEY=your_api_key
# Alternatively pass api_key=your_api_key to the Call class
from dotenv import load_dotenv

load_dotenv()

True

In this notebook we will use free tiers for Moonshot AI's Kimi K2 and Google's Gemma 3N. 

An overview of all available models can be found by calling `get_all_models`:
```python
from irouter.base import get_all_models
model_slugs = get_all_models()
model_slugs
```

In [2]:
model_names = ["moonshotai/kimi-k2:free", "google/gemma-3n-e2b-it:free"]

test_message = "Who played the guitar solo on Steely Dan's Kid Charlemagne?"

# Single Model

The simplest way to use `Call` is use a single LLM by giving a model slug.

In this example we initialize a `Call` object with the free tier of Moonshot AI's Kimi-K2 LLM.

To set the API key you can either set an environment variable for `OPENROUTER_API_KEY` to your project or pass `api_key` when initializing `Call`.

In [3]:
c = Call(model_names[0])
# or
# c = Call(model_names[0], api_key="your_api_key")

In [4]:
nb_markdown(c(test_message))

Larry Carlton played the guitar solo on Steely Dan's *"Kid Charlemagne."*

If needed you can get the raw `ChatCompletion` object from a call by setting `raw=True`.

In [5]:
c(test_message, raw=True)

ChatCompletion(id='gen-1754058132-3A2sjYB7fgN9jmxvtU81', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The guitar solo on Steely Dan\'s *"Kid Charlemagne"* was played by **Larry Carlton**.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1754058132, model='moonshotai/kimi-k2:free', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=26, total_tokens=49, completion_tokens_details=None, prompt_tokens_details=None), provider='Parasail')

## Multiple Models

Calling multiple models at once is as simple as giving a list of model slugs to `Call`.

In [6]:
mc = Call(model_names)

If multiple models are giving a call will return a dictionary mapping with LLM slug and response for easy parsing.

In [7]:
resps = mc(test_message)
resps

{'moonshotai/kimi-k2:free': 'The guitar solo on **"Kid Charlemagne"** by **Steely Dan** was played by **Larry Carlton**.',
 'google/gemma-3n-e2b-it:free': 'The guitar solo on Steely Dan\'s "Kid Charlemagne" was played by **David Spengler**. \n\nHe was a renowned guitarist known for his sophisticated and jazzy style, and he was a frequent collaborator with Steely Dan. This particular solo is considered one of his best and is a highlight of the song.\n\n\n\n'}

It can then be parsed.

In [8]:
nb_markdown(resps["google/gemma-3n-e2b-it:free"])

The guitar solo on Steely Dan's "Kid Charlemagne" was played by **David Spengler**. 

He was a renowned guitarist known for his sophisticated and jazzy style, and he was a frequent collaborator with Steely Dan. This particular solo is considered one of his best and is a highlight of the song.





# Passing messages 

For certain cases you might want to pass a list of messages to the Call class. For example to simulate a conversation.

In that case messages should be defined as a **list of dicts which define `role` and `content`**.

NOTE: The `system` parameter will be ignored if you pass a list of messages. Therefore when using this approach optionally also define a system message in your list of messages.

Here is an example of a conversation between a user and an assistant with a system message at the start.

In [10]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Amsterdam."},
    {
        "role": "user",
        "content": "No silly, why would you say Amsterdam is the capital of France?",
    },
]

c = Call(model_names[0])
c(messages)

"You're absolutely right—I misspoke. The capital of France is Paris. Amsterdam is the capital of the Netherlands."

I hope this gives you a good overview of how to work with `Call`. If you would like to track history and token usage check out the `Chat` object in `irouter`.